<a href="https://www.kaggle.com/code/milkbottledude/titanical-machine-learning-mark-ii?scriptVersionId=207881741" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# Data Configuration


In [2]:
# Train Data Configuration
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data = train_data.drop(['Name', 'Parch', 'PassengerId', 'Ticket'], axis=1)
age_mean = train_data['Age'].mean()
train_data['Age'] = train_data['Age'].fillna(age_mean)
X = pd.get_dummies(train_data.drop('Survived', axis=1))
y = train_data["Survived"]
print(X)

     Pclass        Age  SibSp     Fare  Sex_female  Sex_male  Cabin_A10  \
0         3  22.000000      1   7.2500       False      True      False   
1         1  38.000000      1  71.2833        True     False      False   
2         3  26.000000      0   7.9250        True     False      False   
3         1  35.000000      1  53.1000        True     False      False   
4         3  35.000000      0   8.0500       False      True      False   
..      ...        ...    ...      ...         ...       ...        ...   
886       2  27.000000      0  13.0000       False      True      False   
887       1  19.000000      0  30.0000        True     False      False   
888       3  29.699118      1  23.4500        True     False      False   
889       1  26.000000      0  30.0000       False      True      False   
890       3  32.000000      0   7.7500       False      True      False   

     Cabin_A14  Cabin_A16  Cabin_A19  ...  Cabin_F G73  Cabin_F2  Cabin_F33  \
0        False      

In [3]:
# Test Data Configuration
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
PassengerId_Series = test_data.PassengerId
test_data.drop(['Name', 'Parch', 'PassengerId', 'Ticket'], axis=1, inplace=True)
test_age_mean = test_data['Age'].mean()
fare_median = test_data['Fare'].median()
test_data['Age'] = test_data['Age'].fillna(test_age_mean)
test_data['Fare'] = test_data['Fare'].fillna(fare_median)
X_test_fr = pd.get_dummies(test_data)
print(X_test_fr)

     Pclass       Age  SibSp      Fare  Sex_female  Sex_male  Cabin_A11  \
0         3  34.50000      0    7.8292       False      True      False   
1         3  47.00000      1    7.0000        True     False      False   
2         2  62.00000      0    9.6875       False      True      False   
3         3  27.00000      0    8.6625       False      True      False   
4         3  22.00000      1   12.2875        True     False      False   
..      ...       ...    ...       ...         ...       ...        ...   
413       3  30.27259      0    8.0500       False      True      False   
414       1  39.00000      0  108.9000        True     False      False   
415       3  38.50000      0    7.2500       False      True      False   
416       3  30.27259      0    8.0500       False      True      False   
417       3  30.27259      1   22.3583       False      True      False   

     Cabin_A18  Cabin_A21  Cabin_A29  ...  Cabin_F E46  Cabin_F E57  \
0        False      False   

Gonna try to fix the problem of missing ticket and cabin column names between the testing and training datasets below

In [4]:

SetX = set(X.columns)
SetTestX = set(X_test_fr.columns)
ToRemove = list(SetTestX - SetX)
# print(ToRemove)
ToAdd = list(SetX - SetTestX)
X_test_fr.drop(ToRemove, axis=1, inplace=True)
ToAdd_df = pd.DataFrame(False, index=X.index, columns=ToAdd)
X_test_fr = pd.concat([X_test_fr, ToAdd_df], axis=1)
X_test_fr = X_test_fr.iloc[:418]

# Get the column order of X
column_order = X.columns
# Reordering X_test_fr to match the column order of X
X_test_fr = X_test_fr[column_order]
rows_with_nan = X_test_fr[X_test_fr.isna().any(axis=1)]

# RandomForestClassifier

In [5]:
# from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

model = RandomForestClassifier(random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


# USING GRIDSEARCH
# params = {
#     'n_estimators': [100, 200, 300],           # Number of trees in the forest
#     'max_depth': [None, 10, 20, 30],           # Maximum depth of the trees
#     'min_samples_split': [2, 5, 10],            # Minimum number of samples required to split an internal node
#     'min_samples_leaf': [1, 2, 4],              # Minimum number of samples required to be at a leaf node
#     'bootstrap': [True, False]                  # Whether to use bootstrap samples
# }

# # grid_search = GridSearchCV(
# #     model,
# #     params,
# #     cv=5,  # Number of cross-validation folds
# #     n_jobs=-1,  # Use all available processors
# #     verbose=1,  # Increase verbosity to see the progress
# #     scoring='accuracy'  # Metric to evaluate the model
# # )

# # # Fit the model
# # grid_search.fit(X_train, y_train)

# # Print best parameters and best score
# print("Best Parameters:", grid_search.best_params_)
# print("Best Score:", grid_search.best_score_)

# # Get the best model
# best_model = grid_search.best_estimator_




model.fit(X_train, y_train)
predictions = model.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, predictions))



# scores = cross_val_score(model, X_train, y_train, cv=10)
# print(scores.mean())
# model.fit(X_train, y_train)
# test_score = model.score(X_test, y_test)
# print(test_score)
# mean = test_score/2 + scores.mean()/2
# print(round(mean, 6))

# Best Scores varying dropped columns
# 1 -- RFC: 0.837517   Details: dropped 'Name'
# 2 --RFC: 0.838916  Details: dropped 'Name', 'Parch'
# 3 --RFC: 0.838225   Details: dropped 'Name', 'Parch', 'Cabin'
# 4 --RFC: 0.822852   Details: dropped 'Name', 'Parch', 'Ticket'
# Selecting 2nd variation
#Nevermind i gotta get rid of cabin and ticket its messing with my predictions im not sure why
#Edit: managed to somewhat solve this problem with the 4th cell above

# Best scores varying hyperparameters
# Best Parameters: {'bootstrap': False, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
# Best Score: 0.83006993006993
# Test Accuracy: 0.8491620111731844

Test Accuracy: 0.8491620111731844


# Logistic Regression (TBC)

In [6]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import cross_val_score
# import matplotlib.pyplot as plt
# from sklearn.model_selection import GridSearchCV


# model = LogisticRegression(random_state=0)
# model.fit(X, y)


# Submission

In [7]:
model = RandomForestClassifier(bootstrap=False, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100)
model.fit(X, y)
pred = model.predict(X_test_fr)


# Submission
output = pd.DataFrame({'PassengerId': PassengerId_Series, 'Survived': pred})
print(output)
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         1
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]
Your submission was successfully saved!
